In [28]:
import pickle
import statistics
from sklearn import datasets
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score

In [29]:
filepath = 'e_yrsvp'
file = open(filepath, 'rb')
event_yrsvp = pickle.load(file)
file.close()

filepath = 'category_event_topic_vector_nnmf'
file = open(filepath, 'rb')
category_event_topic_vector = pickle.load(file)
file.close()

# creating the dict necessary for calculation first
category_wise_attendance_dict = {}
for category,event_topic_vector in category_event_topic_vector.items():
    print("category:",category)
    if category == 24:
        continue
    category_wise_attendance_dict[category] = []
    attendance_list = []
    less_attendance_vectors = []
    great_attendance_vectors = []
    less_similarity = 0.0
    less_count = 0
    great_similarity = 0.0
    great_count = 0
    cross_similarity = 0.0
    cross_count = 0
    for event in event_topic_vector.keys():
        try:
            attendance_list.append(event_yrsvp[event])
        except:
            pass
    attendance_median = statistics.median(attendance_list)
    # print("attendance_median:",attendance_median, " len  of attendance list", len(attendance_list))
    for event,topic_vector in event_topic_vector.items():
        try:
            c_attendance = event_yrsvp[event]
            temp_dict = {}
            temp_dict['vector'] = topic_vector
            if c_attendance <= attendance_median:
                less_attendance_vectors.append(topic_vector)
                temp_dict['attendance'] = 0
                
            else:
                great_attendance_vectors.append(topic_vector)
                temp_dict['attendance'] = 1
            category_wise_attendance_dict[category].append(temp_dict)
        except:
            pass
filename = 'category_wise_attendance_dict'
outfile = open(filename,'wb')
pickle.dump(category_wise_attendance_dict,outfile)
outfile.close()

('category:', 0)
('category:', 1)
('category:', 2)
('category:', 3)
('category:', 4)


In [32]:
import numpy as np
from sklearn.decomposition import PCA
import pylab as pl
for category in range(5):
    print('\n')
    X = []
    labels = []
    for event in category_wise_attendance_dict[category]:
        X.append(event['vector'])
        labels.append(event['attendance'])
    X = np.array(X)
    print(labels[:10])
    labels = np.array(labels)
    print('X shape',X.shape)
    print('labels shape',labels.shape)
    pca = PCA(n_components=2).fit(X)
    pca_2d = pca.transform(X)
    
    print('category:',category)
    pl.figure(figsize=(8.0, 5.0))
    pl.scatter(pca_2d[:, 0], pca_2d[:, 1], c=labels)
    pl.savefig('kmeans_results/' + 'actualData_category_' + str(category) + '.png')
    pl.close() 
    
    mxnum_clusters = -1.0
    mxdavies = -1.0
    for num_clusters in range(2,11):
        kmeans = KMeans(n_clusters=num_clusters, random_state=1).fit(X)
        plabels = kmeans.labels_
        score = davies_bouldin_score(X, plabels)
        if mxdavies <= score:
            mxdavies = score
            mxnum_clusters = num_clusters
            mxplabels = plabels
    print("max davies:",mxdavies, " for number of clusters:", mxnum_clusters)
    pl.figure(figsize=(8.0, 5.0))
    pl.scatter(pca_2d[:, 0], pca_2d[:, 1], c=mxplabels)
    pl.savefig('kmeans_results/' + 'category_' + str(category) + '_numclusters' + str(mxnum_clusters)  + '.png')
    pl.close()
        
        
        
    



[1, 1, 1, 1, 0, 1, 0, 1, 0, 0]
('X shape', (20276, 50))
('labels shape', (20276,))
('category:', 0)
('max davies:', 2.2945991735896683, ' for number of clusters:', 9)


[0, 0, 0, 0, 0, 0, 1, 0, 1, 1]
('X shape', (6903, 50))
('labels shape', (6903,))
('category:', 1)
('max davies:', 2.4513031027207015, ' for number of clusters:', 5)


[1, 0, 1, 0, 0, 0, 1, 1, 0, 0]
('X shape', (18799, 50))
('labels shape', (18799,))
('category:', 2)
('max davies:', 2.3756007281921927, ' for number of clusters:', 6)


[1, 1, 0, 0, 0, 1, 0, 1, 0, 0]
('X shape', (2033, 50))
('labels shape', (2033,))
('category:', 3)
('max davies:', 1.5838930019847857, ' for number of clusters:', 5)


[1, 1, 1, 0, 0, 0, 0, 1, 1, 0]
('X shape', (7172, 50))
('labels shape', (7172,))
('category:', 4)
('max davies:', 1.528187159327843, ' for number of clusters:', 8)
